#  手書き認識文字モデルのデモンストレーション

In [ ]:
# 画像や前処理周りのimport
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
from tqdm import tqdm_notebook as tqdm
import pandas as pd
%matplotlib inline

from sklearn.model_selection import train_test_split

# 深層学習周りのimport
import keras
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import np_utils
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# バージョン確認
import matplotlib
import sklearn
import tqdm as tm
print("numpy => {}".format(np.__version__))
print("matplotlib => {}".format(matplotlib.__version__))
print("pandas => {}".format(pd.__version__))
print("OpenCV => {}".format(cv2.__version__))
print("tqdm => {}".format(tm.__version__))
print("scikit-learn => {}".format(sklearn.__version__))
print("keras => {}".format(keras.__version__))

## データセット文字コード対応表の読み込み
ひらがなと漢字の2つの対応表を読み込みます。

In [ ]:
df_hi = pd.read_csv("../data/hiragana_table.csv")

In [ ]:
df_hi.head()

In [ ]:
df_ka = pd.read_csv("../data/kanji_table.csv")

In [ ]:
df_ka.head()

今回のテストに使用するデータを読み込んでいきます。

In [ ]:
def sampling(chars, dir_path, df):
    img_list = []
    label_list = []
    using_dirs = [df[df["char"] == c]["dir"].values[0] for c in chars]
    for using_dir in tqdm(using_dirs):
        imgs_path = os.path.join(dir_path, using_dir)
        img_list.append(cv2.imread(os.path.join(imgs_path, os.listdir(imgs_path)[0])))
        label_list.append(df[df["dir"] == using_dir].index[0])
    
    return img_list, label_list

In [ ]:
using_hiraganas = "そふとこんぴゆうていんぐ"
hi_dir_path = "../data/hiragana73"

hi_imgs, hi_labels = sampling(using_hiraganas, hi_dir_path, df_hi)

In [ ]:
fig, ax = plt.subplots(1, len(hi_imgs), figsize=(18,6))
for i, img in enumerate(hi_imgs):
    ax[i].imshow(img)
    ax[i].axis("off")
plt.show()
print(hi_labels)

In [ ]:
using_kanjis = "一九東南西北白發中國士無又又"
ka_dir_path = "../data/kanji300"

In [ ]:
ka_imgs, ka_labels = sampling(using_kanjis, ka_dir_path, df_ka)

In [ ]:
fig, ax = plt.subplots(1, len(ka_imgs), figsize=(18,6))
for i, img in enumerate(ka_imgs):
    ax[i].imshow(img)
    ax[i].axis("off")
plt.show()
print(ka_labels)

## データ前処理

学習に使用した画像に前処理を行った人は、関数を置き換えて実行してください。

In [ ]:
def preprocessing(img_list):
    img_prop_list = []
    flag = False
    for img in tqdm(img_list):
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # グレースケール化
        img_resize = cv2.resize(img_gray, (28,28)) # リサイズ
        _, img_prop = cv2.threshold(img_resize, 0, 255, \
                                    cv2.THRESH_BINARY + cv2.THRESH_OTSU) # 2値化

        if flag == False: 
            # 初回のみplot
            fig, ax = plt.subplots(1, 3, figsize=(18,8))
            titles = ["gray_scaled", "resized", "binarized"]
            imgs = [img_gray, img_resize, img_prop]
            for n, (k, v) in enumerate(zip(titles, imgs)):
                ax[n].imshow(v, cmap="gray")
                # ax[n].axis("off")
                ax[n].set_title("{}".format(k))
            flag = True

        img_prop_list.append(img_prop)
    return img_prop_list

In [ ]:
hi_imgs = preprocessing(hi_imgs)

In [ ]:
ka_imgs = preprocessing(ka_imgs)

In [ ]:
nb_classes = len(df_hi)
hi_labels = np_utils.to_categorical(hi_labels, nb_classes)

In [ ]:
nb_classes = len(df_ka)
ka_labels = np_utils.to_categorical(ka_labels, nb_classes)

## モデルの読み込みとコンパイル
各自コンパイルの際に使用した最適化関数や誤差関数を変更して実行しましょう

In [ ]:
hi_model = model_from_json(open("../models/hiragana_cnn.json", "r").read())
hi_model.load_weights("../models/hiragana_cnn_best_weight.hdf5")

ka_model = model_from_json(open("../models/kanji_cnn.json", "r").read())
ka_model.load_weights("../models/kanji_cnn_best_weight.hdf5")

In [ ]:
hi_model.compile(loss="categorical_crossentropy", # 誤差(損失)関数
             optimizer="RMSprop", # 最適化関数
             metrics=["accuracy"] # 評価指標
             )
ka_model.compile(loss="categorical_crossentropy", # 誤差(損失)関数
             optimizer="RMSprop", # 最適化関数
             metrics=["accuracy"] # 評価指標
             )

## 学習結果の可視化

In [ ]:
hi_score = hi_model.evaluate(np.array(hi_imgs), hi_labels, verbose=0)
print('Test score:', hi_score[0])
print('Test accuracy:', hi_score[1])

In [ ]:
ka_score = ka_model.evaluate(np.array(ka_imgs), ka_labels, verbose=0)
print('Test score:', ka_score[0])
print('Test accuracy:', ka_score[1])

## 予測の可視化

In [ ]:
hi_tests = np.array(hi_imgs).reshape(len(hi_imgs), 28, 28, 1)
ka_tests =  np.array(hi_imgs).reshape(len(hi_imgs), 28, 28, 1)

In [ ]:
hi_test_pred = hi_model.predict(hi_tests)

In [ ]:
fig, ax = plt.subplots(1, len(hi_imgs), figsize=(18,6))
for i, img in enumerate(hi_imgs):
    ax[i].imshow(img)
    ax[i].axis("off")
plt.show()

In [ ]:
# 予測の可視化
[df_hi.loc[hi_pred.argmax()]["char"] for hi_pred in hi_test_pred]

In [ ]:
ka_test_pred = ka_model.predict(ka_tests)

In [ ]:
fig, ax = plt.subplots(1, len(ka_imgs), figsize=(18,6))
for i, img in enumerate(ka_imgs):
    ax[i].imshow(img)
    ax[i].axis("off")
plt.show()

In [ ]:
# 予測の可視化
[df_ka.loc[ka_pred.argmax()]["char"] for ka_pred in ka_test_pred]